# Co-enhance session 6: 推論

## 環境設定 (Google drive内のファイルへのアクセス)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import sys, os

# チーム番号 (必ず自分のチームにする！)
os.environ['TEAM'] = ''
print('you are team {}'.format(os.environ['TEAM']))

## 学習済みモデルの再利用

In [0]:
root_dir = "./gdrive/Team Drives/coenhance_teams/team{}/content".format(os.getenv('TEAM'))
sys.path.append(os.path.join(root_dir, "code"))

In [0]:
# 深層学習ライブラリpytorch
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

# 可視化モジュール
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import time

# utils.pyから必要な関数などを取ってくる
from utils import *

In [0]:
# GPU使用可能か判定し，deviceに設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU visible: {}".format(torch.cuda.is_available()))

In [0]:
# 使用するモデルのエポック数 (学習時ならば再開するエポック数)
model_num = 100
model_pth = os.path.join(root_dir, "models/ep{}.pth".format(model_num))

# クラスの一覧
food = os.listdir(os.path.join(root_dir, "data"))
food.sort()
print("classes: ", food)

# 使用するニューラルネットワークモデル (上は通常，下は残差ブロックあり)
#net = CNN(image_size=75, num_classes=len(food))
net = CNN_res(image_size=75, num_classes=len(food))

In [0]:
# データ転送 (ある場合GPU)
net = net.to(device)

# 学習済みの重み適用
net.load_state_dict(torch.load(model_pth))

### 再学習

In [0]:
# データ拡張方法
preprocess = transforms.Compose([
    # 360度ランダムで画像を回転する
    transforms.RandomRotation(180),
    # ランダムで上下左右反転する
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    # 明るさやコントラストなどのランダムでの調整
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.2, 0.2, 0.2])
])

In [0]:
# バッチサイズ
BATCH_SIZE = 64
# 学習率
LEARNING_RATE = 1e-2
# ハイパーパラメータβ
BETAS = (0.5, 0.999)

dataset = ImageFolder(os.path.join(root_dir, "data"), transform=preprocess)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collater, num_workers=8)

# 最適化手法 (勾配の谷をどう降りていくか)
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, betas=BETAS)

# 損失関数 (クロスエントロピー)
criterion = nn.CrossEntropyLoss()

In [0]:
# 学習を行うエポック数
NUM_EPOCHS = 100

# モデルを保存するパス
save_path = os.path.join(root_dir, "models")

print('restart training from epoch {}'.format(model_num))
loss_list = []
for ep in range(NUM_EPOCHS):
    
    begin = time.time()
    correct = 0
    full = 0
    running_loss = 0
    cnt = 0
    
    for it, sample in enumerate(dataloader):
        im_batch = sample[0].to(device)
        cl_batch = sample[1].to(device)
        
        optimizer.zero_grad()
        out = net(im_batch)
        correct += torch.sum(torch.argmax(out, dim=1) == cl_batch).item()
        full += BATCH_SIZE
        cnt += 1
        loss = criterion(out, cl_batch)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        running_loss += loss.item()
        
        # 10ステップごとにロスを表示
        if it % 10 == 9:
            print('{}th iter done \t| loss: {}'.format(it+1, loss.item(), flush=True))
        
        
    end = time.time()
    print('-' * 100)
    print('epoch {:03d} \t| loss: {:.05f} \t| train_acc: {:04d}/{:04d} \t| {:.05f}s per loop'.format(model_num + ep + 1, running_loss/cnt, correct, full, (end-begin)/cnt), flush=True)
    print('-' * 100)
    
    # 10エポックごとにモデルを保存
    if ep % 10 == 9:
        path = os.path.join(save_path, 'ep{}.pth'.format(model_num + ep + 1))
        torch.save(net.state_dict(), path)
        print('saved model for epoch {} at {}'.format(model_num + ep + 1, path))
        
print('done!')

## 学習結果の可視化 (損失グラフ，テストデータのCAM)

In [0]:
# 学習損失のイテレーションごとのプロット
plt.plot(loss_list)
plt.title('training loss')
plt.ylabel('cross entropy')
plt.xlabel('iterations')

In [0]:
# 推論データの前処理
preprocess = transforms.Compose([
    transforms.Resize((75, 75)),
    transforms.ToTensor()
])

In [0]:
fig = plt.figure(figsize=(10, 10))
plt.suptitle('visualization of CAM')
net.eval()

# 推論する画像ファイル名(.jpg)
# content/test/にファイルを入れる
filename = 'sushi1.jpg'

path = os.path.join(root_dir, "test/{}".format(filename))

im = Image.open(path)
im_ten_cpu = preprocess(im).unsqueeze(0)
im_ten = im_ten_cpu.to(device)
out = net(im_ten).detach().cpu()
# 普通に分類させる
cl = np.argmax(out)
# 該当したクラスclに関するCAMを取得する
cam = net.get_cam(im_ten, cl)

# 元画像を表示
ax1 = fig.add_subplot(221)
original = im_ten_cpu.squeeze(0).numpy().transpose((1, 2, 0))
plt.imshow(original)
plt.grid(False)
plt.axis('off')
ax1.title.set_text('ground truth: ???')
    
# CAMの乗った画像を表示
ax2 = fig.add_subplot(222)
plt.imshow(cam_on_image(original, cam))
plt.grid(False)
plt.axis('off')
p = out[cl].item() * 100
classname = food[cl]
ax2.title.set_text('predicted class: {}, {:.02f}%'.format(classname, p))